<a href="https://colab.research.google.com/github/lineality/DS-Unit-2-Applied-Modeling/blob/master/module2/Asteroid_v1_GGA_lect_DS8_lesson_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GGAshbrook
Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---

# Permutation & Boosting

- Get **permutation importances** for model interpretation and feature selection
- Use xgboost for **gradient boosting**


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/lineality/asteroid_viz_app_project/master/asteroid_albedo_7.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
!ls

sample_data


## Here we will be making splitting the data into a test-set and a train+validation set (which will then be split into train and val(idation) sets)

In [0]:
import numpy as np
import pandas as pd

#train/validate vs. test split with random seed
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1, random_state=42)
train_val.shape, test.shape

((749634, 10), (83293, 10))

In [0]:
import numpy as np
import pandas as pd

#train validate split with random seed

train, val = train_test_split(train_val, test_size=0.2, random_state=42)
train.shape, val.shape

((599707, 10), (149927, 10))

#Wrangle
##I once had a teacher named Mr. Wrangle...

Here the data will be at first minimally dealt with for initial exploration.

In [0]:



def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    X['diameter'] = X['diameter'].astype(float)
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these values like zero.
#    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # When columns have zeros and shouldn't, they are like null values.
    # So we will replace the zeros with nulls, and impute missing values later.
    # Also create a "missing indicator" column, because the fact that
    # values are missing may be a predictive signal.
#    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
#                       'gps_height', 'population']
#    for col in cols_with_zeros:
#        X[col] = X[col].replace(0, np.nan)
#        X[col+'_MISSING'] = X[col].isnull()
            
    # Drop duplicate columns
#    uplicates = ['quantity_group', 'payment_type']
#    X = X.drop(columns=duplicates)
    
    # Drop recorded_by (never varies) and id (always varies, random)
    #unusable_variance = ['recorded_by', 'id']
#    X = X.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
#    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
#    X['year_recorded'] = X['date_recorded'].dt.year
#    X['month_recorded'] = X['date_recorded'].dt.month
#    X['day_recorded'] = X['date_recorded'].dt.day
#    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
#    X['years'] = X['year_recorded'] - X['construction_year']
#    X['years_MISSING'] = X['years'].isnull()
    
    # return the wrangled dataframe
    return X

train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
# Arrange data into X features matrix and y target vector
target = 'spec_B'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

#Xgboost not in a pipeline

In [0]:
import category_encoders as ce
#from sklearn.impute import SimpleImputer
#from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
#imputer = SimpleImputer()
#scaler = StandardScaler()

In [0]:
model = XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)

In [0]:
X_train_encoded = encoder.fit_transform(X_train)
#X_train_imputed = imputer.fit_transform(X_train_encoded)
#X_train_scaled = scaler.fit_transform(X_train_imputed)
model.fit(X_train_scaled, y_train)

In [0]:
X_val_encoded = encoder.transform(X_val)
#X_val_imputed = imputer.transform(X_val_encoded)
#X_val_scaled = scaler.transform(X_val_imputed)
print('Validation Accuracy', model.score(X_val_scaled, y_val))

In [0]:
X_test_encoded = encoder.transform(X_test)
#X_test_imputed = imputer.transform(X_test_encoded)
#X_test_scaled = scaler.transform(X_test_imputed)
y_pred = model.predict(X_test_scaled)

## Use xgboost for gradient boosting

#### [XGBoost Python API Reference: Scikit-Learn API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)

In [0]:
#from xgboost import XGBClassifier

In [0]:
from xgboost import XGBClassifier

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

In [0]:
#XGBClassifier(n_estimators=100, random_state=42, n_jobs=-1)

In [0]:
pipeline.fit(X_train, y_train)

In [0]:
from sklearn.metrics import accuracy_score
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))